# Classification & Scoring

Classification

In [ ]:
import pandas as pd
import numpy as np
import hvplot.pandas
import matplotlib
import statsmodels.api as sm
import xlsxwriter
import math
from statsmodels.regression.rolling import RollingOLS
import holoviews as hv

#data
data = pd.read_csv('global_equities_PWS_data_24092024.csv', index_col=0, encoding='latin-1')
data1 = data.apply(lambda x: pd.to_numeric(x, errors='coerce'))
data2 = data1.copy()
data2.columns = [data1.columns, data.iloc[0]]
data1.columns = data.iloc[0]
data_cleaned = data1.iloc[1:]

data_cleaned['proposed_portfolio_pws'] = data_cleaned[['Fidelity Asia','Aoris International Fund B',
                                                       'Lazard Global Equity Franchise','Apis Global Long/Short W',
              'Nanuk New World','qus','qsml','qual','vgs']].apply(lambda row: row.mean() if row.notna().all() else np.nan, axis=1)

data_cleaned['proposed_portfolio_pws_vlue'] = data_cleaned[['Fidelity Asia','Aoris International Fund B',
                                                       'Lazard Global Equity Franchise','Apis Global Long/Short W',
              'Nanuk New World','qus','qsml','vlue','vgs']].apply(lambda row: row.mean() if row.notna().all() else np.nan, axis=1)

data_cleaned['proposed_portfolio_pws_dimensional'] = data_cleaned[['Fidelity Asia','Aoris International Fund B',
                                                       'Lazard Global Equity Franchise','Apis Global Long/Short W',
              'Nanuk New World','qus','qsml','dimensional','vgs']].apply(lambda row: row.mean() if row.notna().all() else np.nan, axis=1)

#tracking error
excess_return = data_cleaned.iloc[:,5:].sub(data_cleaned.iloc[:,:1]['Market'], axis=0)
tracking_errors = excess_return.std() * math.sqrt(12)
tracking_errors.index = data_cleaned.iloc[:,5:].columns

#beta estimation
output = {}
output2 = {}
for i in data_cleaned.iloc[:,5:].columns:
    y = data_cleaned[i]
    x = data_cleaned.iloc[:,:5]
    if y.count() < 24:
            output[i] = np.nan
            output2[i] = np.nan
    else: 
        model = sm.OLS(y, sm.add_constant(x), missing='drop').fit()
        output[i] = model.params
        output2[i] = model.tvalues

coefficients = pd.DataFrame(output)
tstats = pd.DataFrame(output2)
coefficients.columns = data_cleaned.iloc[:,5:].columns
tstats.columns = data_cleaned.iloc[:,5:].columns

#classification
classifications = []
for i in tstats.columns:
    ranks = tstats[i].sort_values(axis=0, ascending = False)
    ranks_cleaned = ranks.drop(['Market','const'],axis=0)

    if tracking_errors.loc[i] < 0.01:
        classifications.append('Beta')
        
    elif ranks_cleaned.iloc[0] > 2.581:
        if not (ranks_cleaned.index[-1] == 'Value' and ranks_cleaned.loc['Value'] < -2.581 and (abs(ranks_cleaned.loc['Value']) > ranks_cleaned.iloc[0])):
            classifications.append(ranks_cleaned.index[0])
        else:
            classifications.append('Growth')

    elif (ranks_cleaned.loc['Value'] < -2.581 and (abs(ranks_cleaned.loc['Value']) > ranks_cleaned.iloc[0])):
        classifications.append('Growth')

    elif tracking_errors.loc[i] < 0.02:
        classifications.append('Beta')

    else:
        classifications.append('Alpha')
    

final_df = tstats.T
final_df['Classification'] = classifications
final_df['Tracking Error'] = tracking_errors

#number of months w/ data
masker = excess_return.copy()
masker.columns = data_cleaned.iloc[:,5:].columns
final_df['Number of Months with Data'] = np.where(masker.count(axis=0) > 120, 120, masker.count(axis=0))

"Search" Functionality:

In [ ]:
round(final_df,2)

In [14]:
search = 'nan'
final_df[final_df.index.to_frame().apply(lambda x: x.astype(str).str.contains(search, case=False)).any(axis=1)]

,const,Market,Value,Size,Momentum,Quality,Classification,Tracking Error,Number of Months with Data
apir_ticker,,,,,,,,,
Nanuk New World,-0.029603,17.562774,-0.60304,3.679,-3.005952,3.807282,Quality,0.059132,104


Isolate, wanted:

In [46]:
pws_wanted = ['Aoris International Fund B',
               'Apis Global Long/Short W',
               'Talaria Global Equity','plato']

In [ ]:
use_for_vis = data_cleaned.iloc[:,:5] #.droplevel(axis=1,level=0)
exog = sm.add_constant(use_for_vis, prepend=False)
mod = RollingOLS(df_their_funds['Franklin Global Growth A'], exog, window=36, missing='drop').fit().params['const']
mod1= RollingOLS(df_their_funds['Aoris International Fund B'], exog, window=36, missing='drop').fit().params['const']
mod2 = RollingOLS(df_their_funds['Apis Global Long/Short W'], exog, window=36, missing='drop').fit().params['const']
mod3 = RollingOLS(df_their_funds['Talaria Global Equity'], exog, window=36, missing='drop').fit().params['const']
mod4 = RollingOLS(df_their_funds['Lazard Global Equity Franchise'], exog, window=36, missing='drop').fit().params['const']
mod5 = RollingOLS(df_their_funds['Ironbark Royal London ConcentratedGlbShr'], exog, window=36, missing='drop').fit().params['const']
mod6 = RollingOLS(df_their_funds['Zurich Investments Concentrated Glbl Gr'], exog, window=36, missing='drop').fit().params['const']
mod7 = RollingOLS(df_their_funds['Nanuk New World'], exog, window=36, missing='drop').fit().params['const']

#mod.params.index = pd.to_datetime(mod.params.index)
# plot = mod.params.dropna().iloc[10:,1:-1].hvplot(rot=90, title=f'{fund_wanted}', width=800, height=500, grid=True)
# hline = hv.HLine(0).opts(color='gray',line_width = 0.75)
# combo = plot * hline
# combo

all_mods = pd.concat([mod, mod1, mod2, mod3, mod4, mod5, mod6, mod7], axis=1)
all_mods.columns = ['Franklin Global Growth A', 'Aoris International Fund B', 'Apis Global Long/Short W', 
                    'Talaria Global Equity', 'Lazard Global Equity Franchise', 
                    'Ironbark Royal London ConcentratedGlbShr', 
                    'Zurich Investments Concentrated Glbl Gr', 'Nanuk New World']
all_mods.index = pd.to_datetime(all_mods.index)

In [24]:
df_their_funds = data_cleaned[['Franklin Global Growth A','Aoris International Fund B',
               'Apis Global Long/Short W',
               'Talaria Global Equity','Lazard Global Equity Franchise',
               'Ironbark Royal London ConcentratedGlbShr','Zurich Investments Concentrated Glbl Gr','Nanuk New World']]

In [50]:
final_df['Tracking Error'].loc[pws_wanted]*100

                            apir_ticker               
Aoris International Fund B  Aoris International Fund B     6.578929
Apis Global Long/Short W    Apis Global Long/Short W      10.365203
Talaria Global Equity       Talaria Global Equity          7.403292
plato                       plato                          6.332938
Name: Tracking Error, dtype: float64

In [2]:
excess_return[['Aoris International Fund B',
               'Apis Global Long/Short W',
               'Talaria Global Equity','plato']].corr()

apir_ticker,Aoris International Fund B,Apis Global Long/Short W,Talaria Global Equity,plato
apir_ticker,,,,
Aoris International Fund B,1.000000,-0.125201,-0.196191,-0.021105
Apis Global Long/Short W,-0.125201,1.000000,0.430580,0.205488
Talaria Global Equity,-0.196191,0.430580,1.000000,0.345121
plato,-0.021105,0.205488,0.345121,1.000000


In [51]:
excess_return[['Aoris International Fund B',
               'Apis Global Long/Short W',
               'Talaria Global Equity','plato']].corr()

new_ew = excess_return[pws_wanted].dropna().mean(axis=1)

new_ew.std() * math.sqrt(12)*100

np.float64(5.76562901368139)

In [ ]:
funds_wanted = round(final_df.loc[['Lazard Defensive Australian Equity',
                    'Eiger Australian Small Companies',
                      'Milford Australian Abs Gr W']],2).index

round(final_df.loc[['Lazard Defensive Australian Equity',
                    'Eiger Australian Small Companies',
                      'Milford Australian Abs Gr W']],2).T

Rolling visualization:

In [2]:
fund_wanted = 'global pws'

In [ ]:
use_for_vis = data_cleaned #.droplevel(axis=1,level=0)

exog = sm.add_constant(use_for_vis.iloc[:,:5], prepend=False)
mod = RollingOLS(use_for_vis[fund_wanted], exog, window=36, missing='drop').fit()
mod.params.index = pd.to_datetime(mod.params.index)
plot = mod.params.dropna().iloc[10:,1:-1].hvplot(rot=90, title=f'{fund_wanted}', width=800, height=500, grid=True)
hline = hv.HLine(0).opts(color='gray',line_width = 0.75)
combo = plot * hline
combo

Actual Scoring:

In [ ]:
df_import = pd.read_csv('global_equities_PWS_data_24092024.csv', index_col=0, encoding='latin-1')
data1 = df_import.apply(lambda x: pd.to_numeric(x, errors='coerce'))
data2 = data1.copy()
data2.columns = [data1.columns, df_import.iloc[0]]
data_cleaned = data2.iloc[1:]

new_df = final_df.reset_index()
new_df.index = new_df['apir_ticker']
classification = new_df[['level_0','Classification']]

# df_export = pd.read_csv('new_scoring.csv')
# classification = df_export[['Unnamed: 0', 'apir_ticker', 'Classification']]
# classification.index = classification['apir_ticker']

#beta estimation
output = {}
output2 = {}

data_drop_level_0 = data_cleaned.droplevel(axis=1,level=0)

for i in data_drop_level_0.iloc[:,5:].columns:
    y = data_drop_level_0.loc[:,i].iloc[-120:]
    
    if classification.loc[i]['Classification'] == 'Beta' or classification.loc[i]['Classification'] == 'Alpha':
        x = data_drop_level_0.loc[:,'Market'].iloc[-120:]

    else: 
        classification_value = classification.loc[i]['Classification']
        if classification_value == 'Growth':
            columns_to_select = ['Market', 'Value']
            x = data_drop_level_0.loc[:, columns_to_select].iloc[-120:]
            x['Value'] = -x['Value']
            x = x.rename(columns={'Value':'Growth'})

        else:
            columns_to_select = ['Market', classification_value]
            x = data_drop_level_0.loc[:, columns_to_select].iloc[-120:]

    if y.count() < 24: 
            output[i] = np.nan
            output2[i] = np.nan
        
    else: 
        model = sm.OLS(y, sm.add_constant(x), missing='drop').fit()
        output[i] = model.params
        output2[i] = model.rsquared

coefficients = pd.DataFrame(output)
coefficients.columns = data2.iloc[:,5:].columns
coefficients = coefficients.T
coefs = coefficients.reset_index()
coefs.index = coefs['apir_ticker']
coefs['classification'] = classification['Classification']
coefs = coefs.fillna(0)

#seperate benchmark df and fund returns
benchmark_df = data_drop_level_0.iloc[:,:5]
benchmark_df['Growth'] = -benchmark_df['Value']
fund_returns_df = data_drop_level_0.iloc[:,5:].iloc[-120:]

#align column names
coefs1 = coefs[benchmark_df.drop('Size',axis=1).columns] # coefs[benchmark_df.columns]

#create the monthly alpha with a static beta
bx = pd.DataFrame(index = benchmark_df.iloc[-120:].index, columns = coefs1.iloc[-120:].index)
for j in coefs1.index:
    fund_betas = coefs1.loc[j]
    bx[j] = (benchmark_df.iloc[-120:] * fund_betas).sum(axis=1)

bx1 = bx[fund_returns_df.columns]
final_alpha_df = fund_returns_df - bx1
final_alpha_df.columns = data_cleaned.iloc[:,5:].columns

#std of alpha
std_of_alpha = final_alpha_df.std()*math.sqrt(12)
std_of_alpha1 = pd.DataFrame(std_of_alpha)
std_of_alpha2 = std_of_alpha1.reset_index().set_index('apir_ticker')

#track record and positive months alpha
number_of_months_inception = final_alpha_df.count()
numb_of_pos_months = pd.DataFrame((final_alpha_df>0).sum())
numb_of_pos_months = numb_of_pos_months.reset_index().set_index('apir_ticker')

number_of_months_inception = final_alpha_df.count()
number_of_months_inception1 = pd.DataFrame(number_of_months_inception)
number_of_months_inception1 = number_of_months_inception1.reset_index().set_index('apir_ticker')

classification['Annual-Adjusted-CAPM-Alpha'] = coefs['const'] * 12
classification['Annualized Std of Alpha'] = std_of_alpha2[0]
classification['No. of Positive Months Alpha'] = numb_of_pos_months[0]
classification['No. of Months with Data'] = number_of_months_inception1[0]
classification['Ratio of Positive Months Alpha to Total Months'] = classification['No. of Positive Months Alpha'] / classification['No. of Months with Data']

rounded_classif = round(classification[['Annual-Adjusted-CAPM-Alpha','Annualized Std of Alpha',
                                        'No. of Positive Months Alpha','Ratio of Positive Months Alpha to Total Months']],3)
#add months
rounded_classif['No of Months with Data'] = np.where(masker.droplevel(axis=1,level=0).count(axis=0) > 120, 120, masker.droplevel(axis=1,level=0).count(axis=0))

In [56]:
rounded_classif

,Annual-Adjusted-CAPM-Alpha,Annualized Std of Alpha,No. of Positive Months Alpha,Ratio of Positive Months Alpha to Total Months,No of Months with Data
apir_ticker,,,,,
Pengana Private Equity Trust,0.021,0.118,31,0.484,63
Franklin Global Growth A,-0.039,0.062,53,0.442,120
Aoris International Fund B,-0.015,0.064,37,0.487,75
Zurich Investments Concentrated Glbl Gr,-0.022,0.049,48,0.453,105
Lazard Global Equity Franchise,-0.013,0.061,60,0.500,120
Ironbark Royal London ConcentratedGlbShr,-0.002,0.047,56,0.467,120
Nanuk New World,-0.022,0.059,49,0.467,104
Apis Global Long/Short W,0.009,0.072,55,0.458,120
Fidelity Asia,0.018,0.124,65,0.542,120


In [ ]:
rounded_classif.loc[fund_wanted].T

In [23]:
df2 = data_cleaned[['Ellerston Australian Small-Mid Cap Opportunities', 'Market']].dropna()
df2 = df2.droplevel(axis=1,level=0)
(df2['EllerstonSMID'] - df2['Market']).hvplot(kind='bar',width=850,height=600,rot=90, title = 'Monthly Excess return: Ellerston - ASX300',grid=True)

:Bars   [name]   (0)